### Density Maps for light curves

In [8]:
import pandas as pd
import numpy as np
import zarr
from utils import *
pd.set_option('display.max_columns', 999)

from matplotlib import pyplot as plt
%matplotlib inline

In [7]:
# Run this cell if on SciServer
init()

FileNotFoundError: [Errno 2] No such file or directory: '/home/idies/workspace/Temporary/ywx649999311/LSST_AGN/Class_Training/Data/LCs.zarr.zip'

#### Define Function to Make Density Maps

In [9]:
def dmdt(x, y):
    '''Return dm dt pairs, with dt in the log10 space
    
    Args:
        x(array): An array of time points.
        y(array): The magnitude at each time point in x.
    
    Returns:
        log_dts(array): An array of delta_t between any pair of observations, in
            the log10 scale.
        dms(array): An array of delta_mag corresponding to each delta_t in log_dts.
        dt_amp(float): The largest value in log_dts.
        dm_amp(float): The larges absolute value in dms. 
    '''
    allDiffs = [] # time diff
    allMagDiffs = [] # mag diff
    times = x
    mags = y

    for i in np.arange(1,times.size,1):
        allMagDiffs.append((mags - np.roll(mags, i))[i:])
        allDiffs.append((times-np.roll(times,i))[i:])
        log_dts = np.log10(np.concatenate(allDiffs))
        dms = np.concatenate(allMagDiffs)
    
    dm_amp = np.abs(dms).max() # largest change in mag
    dt_amp = log_dts.max() # largest time interveral
    
    return log_dts, dms, dt_amp, dm_amp


def plotColorDmDt(log_dts, dms, dt_amp, dm_amp, x_bin_size=0.3, y_bin_size=0.002):
    '''Plot the colored DmDt density plot without columne-wise normalization.
    
    Args:
        x(array): An array of time points.
        y(array): The magnitude at each time point in x.
        x_bin_size(float): The size of dt bins, in the log scale.
        y_bin_size(float): The size of dm bins.        
    '''
    
    x_bins = np.int(dt_amp/x_bin_size) # compute number of x bins
    y_bins = np.int(dm_amp/y_bin_size) # compute number of x bins

    # plot
    fig = plt.figure(figsize=(8,6))
    _ = plt.hist2d(log_dts, dms, range=[[0, dt_amp], [-dm_amp, dm_amp]], bins=(x_bins, y_bins))
    plt.colorbar()
    
    return _
    
def plotNormDmDt(denVal, filename=None):
    '''Plot the grayscale DmDt density plot with columne-wise normalization.
    
    Args:
        denVal(array): Density values computed from a colored density map.
        
    '''
    
    fig, ax = plt.subplots(1,1, figsize=(6,6)) # init figure
    
    # perform normalization
    denVal = np.rot90(denVal[0]) # rotate to make x be dt
    y_dim = denVal.shape[0] # number of rows
    x_dim = denVal.shape[1] # number of columns

    # sum over all columns and assign 0 to be 1
    normTotal = np.sum(denVal, axis=0)
    normTotal[normTotal == 0] = 1

    # divide each bin values by normalization and reshape
    normMask = np.repeat(normTotal, y_dim).reshape((x_dim, y_dim)).T
    normd = np.divide(denVal, normMask)

    plt.imshow(np.log(normd), cmap=plt.cm.Greys, extent=[0,3, -1.5, 1.5])
    ax.tick_params(axis='both', which='major', labelsize=15, direction='in', length=5)
    plt.xlabel('$log_{10}(\Delta t\,[days])$', fontsize=15)
    plt.ylabel('$\Delta m \,(arb. unit)$', fontsize=15)
    
    if filename is not None:
        plt.savefig(filename)

### 1. Get Meta info from training data
This section will help find the light curve IDs for objects of different classese (e.g., QSO vs. Star)

In [10]:
## get IDs for variable stars
meta = get_cat('s82Qso')[['train_id', 'class', 'sdss_lcN']]
meta

NameError: name 'meta_data' is not defined

### 2. Variable Stars Density Maps
#### 2.1. Single band 
After obtaining the light curves from the database, you can select which bandpass to plot, that is 'psfmag_u', 'psfmag_g', 'psfmag_r', 'psfmag_i', 'psfmag_z'. Normally, g, r and i bands have the largest number of high quality data. 

In [11]:
# retrive LC by ID and select the data from only a single band
band = 'i'
varlc = get_sdss_lc(1000002)
varlc = varlc.dropna(subset=[f'psfmag_{band}'])

NameError: name 'valid_IDs' is not defined

In [12]:
# make colored density map
log_dts, dms, dt_amp, dm_amp = dmdt(varlc[f'mjd_{band}'].values, varlc[f'psfmag_{band}'].values)
denVal = plotColorDmDt(log_dts, dms, dt_amp, dm_amp, x_bin_size=0.05, y_bin_size=0.01)
plotNormDmDt(denVal)

NameError: name 'varlc' is not defined

#### 2.2 Non-parametric LC merging?
We could try to combine light curves from all five 'ugriz' bands to make a higher resolution density map. 

In [ ]:
varlc = varlc.dropna(subset=['psfmag_g', 'psfmag_u', 'psfmag_r', 'psfmag_i', 'psfmag_z']) # remove nan

In [ ]:
g_log_dts, g_dms, g_dt_amp, g_dm_amp = dmdt(varlc.mjd_g.values, varlc.psfmag_g.values)
r_log_dts, r_dms, r_dt_amp, r_dm_amp = dmdt(varlc.mjd_r.values, varlc.psfmag_r.values)
i_log_dts, i_dms, i_dt_amp, i_dm_amp = dmdt(varlc.mjd_i.values, varlc.psfmag_i.values)

In [ ]:
log_dts = np.append([g_log_dts, r_log_dts], i_log_dts)
dms = np.append([g_dms, r_dms], i_dms)

## catastrophic error could occur if one of the bands has spurious brighness change
dt_amp = np.median([g_dt_amp, r_dt_amp, i_dt_amp])
dm_amp = np.median([g_dm_amp, r_dm_amp, i_dt_amp])

In [ ]:
denVal = plotColorDmDt(log_dts, dms, dt_amp, dm_amp, x_bin_size=0.05, y_bin_size=0.01)
plotNormDmDt(denVal)

### 3. Things to explore
- Change the ID and see how the density map changes, this is how we might be able to classify different classes of stars.
- Change the `x_bin_size` and `y_bin_size` argument for the `plotColorDmDt` function. These two parameters could end up being hyperparameters.
- Many more...

In [ ]:
help(plot_sdss_lc)

In [ ]:
###NEW THINGS START HERE
###Leaving everything above to avoid deleting necessary variables

In [ ]:
denVal = plotColorDmDt(log_dts, dms, dt_amp, dm_amp, x_bin_size=0.05, y_bin_size=0.01)
plotNormDmDt(denVal)